In [1]:
import pymatgen
from pymatgen.ext.matproj import MPRester
from pymatgen.core.structure import Structure
from pymatgen.symmetry.groups import SpaceGroup, SYMM_DATA, sg_symbol_from_int_number

In [2]:
# To make Materials Project queries, you need to have your Materials Project API key set in ~/.pmgrc.yaml with:
# PMG_MAPI_KEY: YOUR_API_KEY
a = MPRester()

In [3]:
# This is a list of the point groups as noted in pymatgen
point_groups = []
for i in range(1,231):
    symbol = sg_symbol_from_int_number(i)
    point_groups.append(SYMM_DATA['space_group_encoding'][symbol]['point_group'])

# Note that there are 40 of them, rather than 32.
print("Number of point groups denoted in pymatgen: ", len(set(point_groups)))

# This is because multiple conventions are used for the same point group.
# This dictionary can be used to convert between them.
point_group_conv = {'321' :'32', '312': '32', '3m1' :'3m', '31m': '3m',
                    '-3m1' : '-3m', '-31m': '-3m', '-4m2': '-42m', '-62m': '-6m2' }

# Using this dictionary we can correct to the standard point group notation.
corrected_point_groups = [point_group_conv.get(pg, pg) for pg in point_groups]
# Which produces the correct number of point groups. 32.
print("Number of point groups in conventional notation: ", len(set(corrected_point_groups)))

Number of point groups denoted in pymatgen:  40
Number of point groups in conventional notation:  32


In [4]:
# polar_point_groups = ['1', '2', 'm', 'mm2', '4', '4mm', '3', '3m', '3m1', '31m','6', '6mm']
# There are 10 polar point groups
polar_point_groups = ['1', '2', 'm', 'mm2', '4', '4mm', '3', '3m', '6', '6mm']

# Polar spacegroups have polar point groups.
polar_spacegroups = []
# There are 230 spacegroups
for i in range(1,231):
    symbol = sg_symbol_from_int_number(i)
    pg = SYMM_DATA['space_group_encoding'][symbol]['point_group']
    if point_group_conv.get(pg, pg) in polar_point_groups:
        polar_spacegroups.append(i)
        
# 68 of the 230 spacegroups are polar.
print("Number of polar spacegroups: ", len(polar_spacegroups))

Number of polar spacegroups:  68


In [5]:
# Create dictionary of nonpolar supergroups of polar spacegroups.
nonpolar_supergroups = {psg : [] for psg in polar_spacegroups}
for psg in polar_spacegroups:
    for i in range(1,231):
        if i not in polar_spacegroups:
            sg = SpaceGroup.from_int_number(psg)
            sg2 = SpaceGroup.from_int_number(i)
            if sg.is_subgroup(sg2):
                nonpolar_supergroups[psg].append(sg2.int_number)

In [6]:
# Now we are ready to make the query.
min_band_gap = 0.0
polar_query = a.query(criteria={'band_gap': {'$gt': min_band_gap},
                                'spacegroup.number': {'$in': polar_spacegroups},
                                # For simplicity (and speed), we use BaTiO3 as an example.
                                # Remove this criteria to search generally. This will take much longer.
                                'pretty_formula': 'BaTiO3'},
                      properties=['material_id', 'spacegroup.number',
                                  'pretty_formula', 'band_gap','nsites'])
print("Number of polar structures in Materials Project: ", len(polar_query))

Number of polar structures in Materials Project:  4


In [7]:
# For each polar insulator in the Materials Project find compatible nonpolar structures
with open('polar_MP.txt','w') as f:
    with open('nonpolar_polar_pairs.txt','w') as f2:
        for p in polar_query: 
            f.write(p['material_id']+"\t"+p['pretty_formula']+"\t"+\
                    str(p['spacegroup.number'])+"\t"+str(p['nsites'])+"\n")
         
            # Query for nonpolar supergroup structures with same composition for polar structures.
            nonpolar_query = a.query(criteria={"spacegroup.number" : 
                {'$in' : nonpolar_supergroups[p['spacegroup.number']] },
                 'pretty_formula' : p['pretty_formula'], 
                 'nsites': {"$lte": p['nsites']}},
                properties = ["material_id", "pretty_formula", "spacegroup.number",
                              "nsites"])
     
            for np in nonpolar_query:
                pair_str = p['material_id']+"\t"+np['material_id']+"\t"+\
                    p['pretty_formula']+"\t"+str(p['spacegroup.number'])+\
                    "\t"+str(np['spacegroup.number'])+"\t"+str(p['nsites'])+\
                    "\t"+str(np['nsites'])+"\n"
                print(pair_str)
                f2.write(pair_str)

mp-5777	mp-19990	BaTiO3	38	123	5	5

mp-5777	mp-504715	BaTiO3	38	221	5	5

mp-5777	mp-2998	BaTiO3	38	221	5	5

mp-5986	mp-19990	BaTiO3	99	123	5	5

mp-5986	mp-504715	BaTiO3	99	221	5	5

mp-5986	mp-2998	BaTiO3	99	221	5	5

mp-644497	mp-19990	BaTiO3	38	123	5	5

mp-644497	mp-504715	BaTiO3	38	221	5	5

mp-644497	mp-2998	BaTiO3	38	221	5	5

mp-5020	mp-504715	BaTiO3	160	221	5	5

mp-5020	mp-2998	BaTiO3	160	221	5	5

